## load packages and read file

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
from nltk.tokenize import RegexpTokenizer

# initiailze tokenizer
tokenizer = RegexpTokenizer(r'\w+')

warnings.filterwarnings(action = 'ignore')
import numpy as np
import gensim
from gensim.models import Word2Vec
import pandas as pd
import collections
import itertools
import nltk
nltk.download('punkt')
import math

from scipy.spatial.distance import cosine


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
xls=pd.ExcelFile('drive/MyDrive/agoda/mappinghotelsdataset.xlsx')
P1 = pd.read_excel(xls, 'Partner1', na_filter=False)
P2 = pd.read_excel(xls, 'Partner2',na_filter=False)

## Prepare  the corpus by tokenizing selected nominal values
#### no need to remove NA's as they are considered as the empty string ('')

In [3]:
p14=[str(x) for x in P1.iloc[:,4]]
p24=[str(x) for x in P2.iloc[:,4]]
p16=[str(x) for x in P1.iloc[:,6]]
p26=[str(x) for x in P2.iloc[:,6]]
p1=P1.iloc[:,1]+" "+ P1.iloc[:,2] +  " " + P1.iloc[:,3]+" "+ " " +p14+" " +p16
p1_a=P1.iloc[:,1]
p1_1=P1.iloc[:,2]# +  " " + P1.iloc[:,3]
p1_A=p14
p2=P2.iloc[:,1]+" "+ P2.iloc[:,2] + " " + P2.iloc[:,3] + " " +p24+" " + p26
p2_a=P2.iloc[:,1]
p2_1=P2.iloc[:,2]
p2_A=p24



def tokenize(P:list):
  temp1 = []

  for p in P:
    
    try: 
      for j in tokenizer.tokenize(p):
        temp1.append(j.lower())
      
    except:
      continue;
  return temp1
words=tokenize(p1+p2)


counter=collections.Counter(words)
mc=[i[0] for i in counter.most_common(100)]

In [4]:
def gather_data(P:list):
  '''
  tokenize data; 
  return tokenized bag of words
  '''
  data = []

  for p in P:
        temp = []
        try:
          for j in tokenizer.tokenize(p):
            if j.lower() not in mc:

              temp.append(j.lower())

          data.append(temp)
        except:
          pass
  return data

data1=gather_data(p1); data2=gather_data(p2); data1_1=gather_data(p1_1); data1_A=gather_data(p1_A)
data2_1=gather_data(p2_1); data1_a=gather_data(p1_a); data2_a=gather_data(p2_a); data2_A=gather_data(p2_A)

## run nltk's word2vec with sg=1 (stands for skip gram of 1 n)

In [5]:
# run the word 2 vec algorithm; cbow method
model1w = gensim.models.Word2Vec(data1+data2, min_count = 1, 
                              size = 250, window = 3,sg=1)


# get embedded representations of words and add to relevant DSs
w2v1=[sum([model1w.wv[y] for y in x]) for x in data1]
w2v2=[sum([model1w.wv[y] for y in x]) for x in data2]
w2v1_1=[sum([model1w.wv[y] for y in x]) for x in data1_1]
w2v2_1=[sum([model1w.wv[y] for y in x]) for x in data2_1]
w2v1_a=[sum([model1w.wv[y] for y in x]) for x in data1_a]
w2v2_a=[sum([model1w.wv[y] for y in x]) for x in data2_a]
w2v1_A=[sum([model1w.wv[y] for y in x]) for x in data1_A]
w2v2_A=[sum([model1w.wv[y] for y in x]) for x in data2_A]


P1["embedded_name"]=w2v1
P2["embedded_name"]=w2v2
P1["embedded_cc"]=w2v1_1
P2["embedded_cc"]=w2v2_1
P1["embedded_name_a"]=w2v1_a
P2["embedded_name_a"]=w2v2_a
P1['embedded_add']=w2v1_A
P2['embedded_add']=w2v2_A
P1['tokenized_name']=data1
P2['tokenized_name']=data2
P1['tokenized_cc']=data1_1
P2['tokenized_cc']=data2_1
P1['tokenized_name_a']=data1_a
P2['tokenized_name_a']=data2_a
P1['tokenized_add']=data1_A
P2['tokenized_add']=data2_A



In [6]:
from copy import deepcopy
P11=deepcopy(P1);P22=deepcopy(P2)

In [7]:
s=set(list(P2["p2.country_code"])+list(P1["p1.country_code"]))

P11.sort_values(by=["p1.country_code"])


,p1.key,p1.hotel_name,p1.city_name,p1.country_code,p1.hotel_address,p1.star_rating,p1.postal_code,embedded_name,embedded_cc,embedded_name_a,embedded_add,tokenized_name,tokenized_cc,tokenized_name_a,tokenized_add
5797,ACFE9A2AA21CFB8C0164FE1FB2077995,Nirvana Holiday Homes - Sky View,Dubai,AE,Jumeirah Beach Road,0.0,,"[0.23985083, 0.1359621, 0.08622184, 0.5520509,...","[0.0906117, 0.018608693, -0.026342249, 0.09616...","[0.07457642, 0.08311945, 0.104099974, 0.333986...","[-0.00030174543, 0.0016875485, -0.0010588607, ...","[nirvana, holiday, homes, sky, view, dubai, ae...",[dubai],"[nirvana, holiday, homes, sky, view]",[jumeirah]
2059,D8770B2F8D4A764DAA3CCACC78777BAE,Dream Inn Dubai Apartments - Al Sahab,Dubai,AE,"Al Sahab, Dubai Marina",0.0,,"[0.37224707, 0.12306656, -0.0416674, 0.5215299...","[0.0906117, 0.018608693, -0.026342249, 0.09616...","[0.091837585, 0.037471153, -0.012115473, 0.136...","[0.114833325, 0.034440294, -0.012732655, 0.171...","[dream, dubai, sahab, dubai, ae, sahab, dubai,...",[dubai],"[dream, dubai, sahab]","[sahab, dubai, marina]"
8768,51E153135BD683C2D19D487E9C4305C6,Sheraton Dubai Mall of the Emirates Hotel,Dubai,AE,"Sheikh Zayed Road, al Barsha 1",5.0,,"[0.31660965, 0.1518788, 0.025974296, 0.5527547...","[0.0906117, 0.018608693, -0.026342249, 0.09616...","[0.12012661, 0.080727085, 0.031943686, 0.26654...","[0.030906908, 0.019996624, 0.010849885, 0.0728...","[sheraton, dubai, mall, emirates, dubai, ae, s...",[dubai],"[sheraton, dubai, mall, emirates]","[sheikh, zayed, barsha]"
9873,C4089B4631846440AAD3808AE6956DEF,The Address Dubai Mall Hotel,Dubai,AE,Emaar Boulevard,5.0,,"[0.33088988, 0.12741263, 0.03345774, 0.5627911...","[0.0906117, 0.018608693, -0.026342249, 0.09616...","[0.11596446, 0.038197342, 0.0017949548, 0.1943...","[0.04934926, 0.038060177, 0.04848206, 0.155055...","[address, dubai, mall, dubai, ae, emaar, boule...",[dubai],"[address, dubai, mall]","[emaar, boulevard]"
2179,55EC13C3E59DBEC7785C92EFDC9D32C4,Al Hayat Hotel Apartments,Sharjah,AE,Al Zahara Street,3.0,,"[0.09497865, 0.0406579, 0.014801091, 0.1694241...","[0.017989594, 0.0069926674, 0.003988882, 0.044...","[0.0031739797, 0.0018889101, 0.0013704855, 0.0...","[-0.0011493713, -0.0007700937, -8.124997e-05, ...","[hayat, sharjah, ae, zahara]",[sharjah],[hayat],[zahara]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,C147FC455D8BDF41123A2E607252A9A7,Villa Afrique,Cape Town,ZA,29 Sluysken Road,5.0,7806,"[0.0103668235, 0.0985159, 0.08481771, 0.226305...","[0.0077384673, 0.039140303, 0.035821587, 0.100...","[0.0034913237, 0.00081797515, -0.00082343, 0.0...","[-0.00035837915, 0.05465962, 0.046279162, 0.11...","[afrique, cape, 29, sluysken, 7806]",[cape],[afrique],"[29, sluysken]"
393,4D73E001C6153EBCD2EF0D777C3A1798,Kelvin Grove Guest House,Addo,ZA,R336 from Addo to Kirkwood,3.0,6105,"[0.050658483, 0.071445115, 0.09650209, 0.28850...","[0.0023399787, 0.002118656, 0.004440095, 0.018...","[0.006914715, 0.016861537, 0.019506522, 0.0525...","[0.038058378, 0.051129173, 0.07228992, 0.21248...","[kelvin, grove, addo, r336, from, addo, to, ki...",[addo],"[kelvin, grove]","[r336, from, addo, to, kirkwood]"
1268,9E240AE993FD0301A9D0C758AD3AACC6,Wild Peach Inn,Pretoria,ZA,"69 Joseph Road, Lynnwood Glen",3.0,81,"[0.041708343, 0.08943835, 0.10351606, 0.292897...","[0.0013812743, 0.012153095, 0.011351916, 0.030...","[0.005901312, 0.0090841865, 0.011110572, 0.030...","[0.026888363, 0.04179813, 0.051839944, 0.15553...","[wild, peach, pretoria, 69, joseph, lynnwood, ...",[pretoria],"[wild, peach]","[69, joseph, lynnwood, glen]"
8516,BD0B7F011E452E8A8B07014623F10763,N1 Hotel & Campsite Victoria Falls,Victoria Falls,ZW,266 Adam Stander Drive,2.0,,"[0.07923187, 0.09460268, 0.115563884, 0.368536...","[0.03653915, 0.036577795, 0.047784425, 0.15115...","[0.038066685, 0.04516484, 0.054440565, 0.16977...","[0.0021005145, 0.008631946, 0.012299973, 0.036...","[n1, campsite, victoria, falls, victoria, fall...","[victoria, falls]","[n1, campsite, vict

## use cosine similaiirity and IoU to detect matches

In [ ]:
%%time
MATCHED_KEYS=[]
p2fields3=[]

for c in s:
  print(c)
  P11c=P11[P11['p1.country_code']==c]
  P22c=P22[P22['p2.country_code']==c]
  
  for i, rowi in P11c.iterrows():
    cos10=[]
    lll=[]
    for j, rowj in P22c.iterrows():
      ll=(len(set(rowi["tokenized_name"]).intersection(rowj["tokenized_name"]))-len(set(rowi["tokenized_name"]).difference(rowj["tokenized_name"])))/len(set(rowi["tokenized_name"]+rowj["tokenized_name"]))
      cs=cosine(rowi[-8], rowj[-8]) -sum([1/counter[w] for w in set(rowi[-4]+rowi[-2]+rowi[-1]).intersection(rowj[-4]+rowj[-2]+rowj[-1])]) 
      

      lll.append(ll)
      cos10.append(cs-ll)

    if lll[cos10.index(min(cos10))]>=0:
      p2fields3=P2[P2['p2.country_code']==c].iloc[cos10.index(min(cos10)),:4]
      MATCHED_KEYS.append((rowi[0],p2fields3[0]))
MK1=pd.DataFrame(MATCHED_KEYS)


CO
AU


## Deal with duplicates

In [ ]:
MK1_c=deepcopy(MK1)
len(MK1_c[1])-len(set(MK1_c[1]))
l=list(MK1_c[1])
dup=set([x for x in l if l.count(x) > 1])

In [ ]:
gl=[]
for d in list(dup):

  l1=[]; l2=[]
  for i in range(len(list(MK1[MK1[1]==d][0]))):
    l1.append(sum(cosine([P11[P11['p1.key']==r] for r in list(MK1[MK1[1]==d][0])][i]["embedded_name"],P22[P22['p2.key']==d]["embedded_name"])))
    l2.append([P11[P11['p1.key']==r] for r in list(MK1[MK1[1]==d][0])][i].iloc[0][0])
  gl.append((l1,l2))


In [ ]:
for i in gl:
  idx=(i[0].index(min(i[0])))
  if min(i[0])<0.5:
    for k in range(len(i[0])):
      if k!=idx:
        MK1_c=MK1_c[MK1_c[0] != i[1][k]]
  else:
    for k in range(len(i[0])):
      MK1_c=MK1_c[MK1_c[0] != i[1][k]]


## save results file as mappings.csv

In [ ]:
MK1_c.to_csv('drive/MyDrive/agoda/mappings.csv', index=False)  